# Data analysis of the models results

This notebook aims to provide analytical tools to work on the damage functions modeled in the WILIAM model folder. 

In [1]:
import pysd
import pandas as pd

In [2]:
reload = True

if reload:
    model = pysd.read_vensim(r'C:\\Users\\gabriel.genelot\\Documents\\damage-functions\\WILIAM_v1.3\\WILIAM.mdl')
else:
    pass

c:\Users\gabriel.genelot\AppData\Local\Programs\Python\Python312\Lib\site-packages\pysd\builders\python\subscripts.py:337: UserWarning: 
Dimension given by subscripts:
	{'HIGH EDUCATION', 'MEDIUM EDUCATION'}
is incomplete using EDUCATIONAL LEVEL I instead.
Subscript_dict:
	{'A MATRIX YEARS I': ['year1995', 'year1996', 'year1997', 'year1998', 'year1999', 'year2000', 'year2001', 'year2002', 'year2003', 'year2004', 'year2005', 'year2006', 'year2007', 'year2008', 'year2009'], 'AEZ I': ['AEZ TROPICAL', 'AEZ TEMPERATE', 'AEZ ARID', 'AEZ BOREAL'], 'AGE ADULTS I': ['c15c19', 'c20c24', 'c25c29', 'c30c34', 'c35c39', 'c40c44', 'c45c49', 'c50c54', 'c55c59', 'c60c64'], 'AGE CHAIN I': ['c5c9', 'c10c14', 'c15c19', 'c20c24', 'c25c29', 'c30c34', 'c35c39', 'c40c44', 'c45c49', 'c50c54', 'c55c59', 'c60c64', 'c65c69', 'c70c74', 'c75c79', 'cover80'], 'AGE CHAIN MIDDLE I': ['c5c9', 'c10c14', 'c15c19', 'c20c24', 'c25c29', 'c30c34', 'c35c39', 'c40c44', 'c45c49', 'c50c54', 'c55c59', 'c60c64', 'c65c69', 'c70c74'

NotImplementedError: _ext_constant_fund_a2_alpha_parameter
The files with extension .csv are not implemented